Gaëlle Loutfi \n


<h1 style="text-align:center; font-size:40px">Coding Project - PetitJean Index </h1>

***


    
* The Petitjean index is a popular topological descriptor, which is a type of shape descriptor that is calculated from the distance matrix of a molecular structure.

* This index uses the longest through-graph distance between each atom in the structure using a variant of the all-pairs shortest path algorithm.

* Petitjean defined the eccentricity of an atom in a molecular structure as the longest path between that atom and any other atom in the structure.
    * radius (R) of a molecular structure as the smallest atom eccentricity
    * diameter (D) as the largest eccentricity



The Petitjean Index can simply be calculated by the equation below: 

### I = (D-R)/R

***

<font color=green size=5px>**Part 1**</font>
<div style='text-align:justify; font-size:16px'>



First of all, we need to extract the necessary information from the molfiles. \
For this purpose, we will define the function `extract_mol()`\
In this function we will proceed sequentially, in the following way:
* First, we need to initialize an array, that we will name "bonds". 
* We will use this array to store pairs of integers from the molfile, these will represent the atom numbers which are connected by a bond. 
* Next, we initialize an empty string **Id** that will be later used to store the ID of the molecule, retrieved from CHEMBL.


Now, we can open the molfile and start reading.
* Each line in the molfile will be split into an element of the array **lines** using `splitlines()`
*NB: The number of bonds and number of atoms are found in the 4th line of every molfile, at indices 1 and 0 respectively.
* We initialize 2 variables **bondCount, atomCount** to store the number of bonds and atoms respectively.

Next, we need a "for loop:", which will loop over each element in the **lines** array, representing the lines of the file.\
Then, each element of the array will be consequently split into another array, based on the position of the whitespaces (which is why we used .strip().split()).\
*Rq: we need to make sure that the first 3 lines are ignored, as well as the blank lines.

Now, onto line 4 (which is the element number 3 in the **lines** array, as indexing starts from 0):
* this line contains the number of atoms and bonds, both of which will be recorded in their respective variables, which we previously set to 0. 
* This function adds tuples containing the pairs of numbers found in the bonds section of the file. 
* All there is left to do is check if the line starts with the string "CHEMBL", which normally comes right before the ID of the molecule. If the string is found, we can now extract the ID and store it in its corresponding variable. 

After everything is properly executed, the function will return the bonds array, as well as the CHEMBL ID, and the atoms count value. 


In [ ]:
def extract_mol():
    bonds=[]  
    Id='' 
    mol_file_path=input("Enter your mol file path:")
    
    with open(mol_file_path, 'r') as mol_file:
        lines=mol_file.read().splitlines() 
                                           
        bondCount=0   
        atomCount=0
        
        for i in range(len(lines)):
            elements=lines[i].strip().split()  
            if(i==3):  
                atomCount=int(elements[0].strip())
                bondCount=int(elements[1].strip())
            elif(i>3+atomCount and i<=3+atomCount+bondCount):
                bonds.append((int(elements[0]),int(elements[1])))  
            elif lines[i].startswith("CHEMBL"): 
                Id=lines[i]
            
    return bonds,Id,atomCount

Next, we will be constructing a graph using the networkx library, that we imported as nx for brevity purposes. 

(NetworkX is a Python library for studying graphs and networks.)

We defined a `construct_graph(bonds)` function, which takes the bonds list, returned by the previous function, as an input. 
* First, this function creates an empty undirected graph **"G"** using "nx.Graph()" 
* Next, it will add edges to the graph G based on the input list of bonds. This will be done using the the "add_edges_from" method. 

Finally, the function returns the constructed graph G.





In [ ]:
import networkx as nx

def construct_graph(bonds):
    G=nx.Graph()
    G.add_edges_from(bonds)
    return G

Before moving on to the following cell, it is important to note that while we already have imported networkx, for this function to work it is also necessary to import numpy as well. 

(NumPy is a library for the Python programming language. Used for adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.)

Now we define the function `shortest_path(G, atomCount)`, which we will use to compute the shortest path lengths between all pairs of nodes (representing the atoms) in the graph. This function will return a 2D matrix representing these lengths. 

This function takes 2 inputs:
1. G, which is the networkx graph returned through the previously defined `construct_graph(bonds)` function. 
1. atomCount, the number of atoms returned from `extract_mol()`


* The function will start by initializing a 2D matrix filled with zeroes. The Dimensions of this matrix will be **atomCount by atomCount**
* Next, we have 2 for loops:
    - in the outer one, i ranges from one to atomCount, which is the number of atoms, inclusive. 
    - in the inner one, j has the same range. 
* these nested for loops will be used to iterate over the pairs of nodes (i,j) representing any 2 atoms.
* while iterating over these pairs of atoms, it will calculate the shortest path length between the 2 atoms involved, using **nx.shortest_path** 
* it is important to note that we subtract 1 from the the computed length of the shortest path as the nx.shortest_path() function returns a list of nodes from the source to the target(inclusive) and the number of nodes is 1 more than the number of edges.
* Each "shortest path length" will be stored in the corresponding cell of the matrix. 

After the entire matrix **matrix** is filled, we will use numpy.delete to remove the first row and column of the matrix as no atoms are numbered 0 and the value at those indices remains 0.

After all is done, the function will return the matrix containing the shortest path length between the atoms. 



In [ ]:
import numpy

def shortest_path(G,atomCount):
    matrix=[[0]*atomCount for _ in range(atomCount)]
    for i in range(1,atomCount):
        for j in range(1,atomCount):
            shortestPathLen=len(nx.shortest_path(G,source=i,target=j))-1
            matrix[i][j]=shortestPathLen
    
    matrix = numpy.delete(matrix, (0), axis=0) 
    matrix = numpy.delete(matrix, [0], 1)          

    return matrix


In the following cell, we are looking to calculate the eccentricities. For this purpose we have defined a function `longest_shortest_paths(matrix)`, that takes a matrix as an input. Note that the matrix used as input in this function is the one returned by the`shortest_path(G,atomCount)` function we have defined in the previous cell. 

*The purpose of this function is to compute the eccentricity of each node in the graph (the longest shortest path of every atom).*

Back to our function. As we previously mentioned, this function takes a matrix of the form **matrix[i][j]** as an input. This matrix represents the shortest path length between two nodes/atoms **i** and **j**.

The function begins by initializing a variable N, which corresponds to the length of the input matrix. This length refers to the number of nodes in the graph.

Next, we will initialize the list **eccentricities**. All of its elements will be initially set to 0. We will consequently be using this list to store the eccentricities of each node/atom. 

Now onto the nested *for loop* of range N (goes from 0 to N-1):
* Inside the outer loop, we will initalize a variable **maxx** to 0. We will be using it subsequently to track the maximum shortest path from a node i to any other node in the graph. 
* The inner loop iterates over all the other nodes j, and for each pair of i and j nodes, we comapre matrix[i][j] to maxx.
* This is done to update the value of **maxx** with the maximum shortest path length for a certain i node.
* Finally, we will assign the value of **maxx** as the eccentricity of the node i in the eccentricities list. 

When all of this is done, after iterating through all the nodes, the function will return the array **eccentricities**, which contains the eccentricity of each node(atom) in the graph represented by the original input matrix. 

In [ ]:
def longest_shortest_paths(matrix):
    
    N=len(matrix)
    eccentricities = [0]*N
    for i in range(N):
        maxx=0
        for j in range(N):
            maxx=max(matrix[i][j],maxx)
        eccentricities[i]=maxx
    return eccentricities


Now onto the following cell. We have defined a function `find_D_R(eccentricities)`, which takes the **eccentricities** array as an input in order to find the *Diameter* **D** and the *Radius* **R**, which have already been mentioned in the introduction of the topic. \
Nevertheless, to reemphasize on their importance in calculating the petitjean index:
* The Diameter will represent the largest atom eccentricity among all nodes in the graph, which is the maximum longest shortest path distance between any 2 atoms in the graph.
* The Radius of the molecular structure will be the smallest or minimum eccentricity among all the atoms, hence the minimum longest shortest  path in the graph.
* These are the main 2 components of the formula needed to calculate the petitjean index, which we will cover in the next cell. 

By applying the **max** and **min** functions on the **eccentricities** list, our defined function will simply return the Diameter and the Radius. 

In [ ]:
def find_D_R(eccentricities):
    D=max(eccentricities)
    R=min(eccentricities)
    return D,R

In Order to create the Graphical User Interface used to display our molecules, we have implemented the following: 

First of all, let us state the purpose of our imported libraries and functions:
* **tkinter** is used to create the GUI. 
* **matplotlib.pyplot** is used to create plots and figures.
* **FigureCanvasTkAgg** is used to embed a Matplotlib Figure into a Tkinter canvas. In other words, we are using it to integrate the Matplotlib's plotting capabilities into our graphical user interface. 

After importing what is needed, we will create the function that will be used to create this project's GUI. The function will be defined as `GUI(G,D,R,Id,PetitJeanIndex)`. \
This function, implemented in the following cell, takes as input the following parameters:
1. The graph *G* returned from the `construct_graph(bonds)` function.
1. The diameter *D* and radius *R* returned from the `find_D_R(eccentricities)` function. 
1. The ID *Id* returned from the `extract_mol()` function. 
1. The Petitjean index *PetitJeanIndex* that we will later calculate using the previously mentioned formula. 

In the function, we first start by creating a Tkinter root window of size 800x600 pixels, and set the title of this window as "PetitJean Index". The molecule representation will be displayed in this window. 

Now, we move on to creating the label widget. The text is based on the "Id" variable. We are using an Arial Bold font. Then we pack it into the GUI. We set **padx** and **pady** to 20 as it adds 20 pixels from the right, left, top and bottom of the label.

Next we will create another label. This one will be based on the values of *D*, *R* and the *PetitJeanIndex*. This label has a slightly smaller Arail font than the previous one. It will be packed into the root window as well. 

Now, a Matplotlib figure will be created with a specified size of 15 by 15. In order to compute the positions of the nodes (atoms) in the graph, the Kamada-Kawai layout from NetworkX will be used, which we found to be the most suitable for our purpose. The scale parameter is set to 5, this adjusts the spacing between the nodes.
 
*The Kamada-Kawai layout is an algorithm that attempts to position the nodes in a graph in such a way to minimize the total energy of the system, as in: connected nodes are close, and unrelated nodes are distant. This creates a visually pleasing representation of the graph*.


Then, using the **nx.draw** function, we will present the graph with the node postions computed by the layout algorithm on the Matplotlib figure. The nodes in this graph will be unlabeled, colored in blue, and of size 500. 

A **FigureCanvasTkAgg** object will be created taking as input **fig**, the MAtplotlib figure, and **root**, the tkinter root window, both of which we have previously gotten. This is used to embed the Matplotlib figure into the GUI. 

Now, we just pack the canvas widget into the Tkinter GUI, in order to display the Matplotlib figure (representing the molecule) on the Tkinter window. 

Finally, the Tkinter GUI is displayed using the main loop. 


In [ ]:
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

def GUI(G,D,R,Id,PetitJeanIndex):
    root = tk.Tk()
    root.geometry("800x600")
    root.title("PetitJean Index")
    label=tk.Label(root,text=Id,font=("Arial",20,'bold'))
    label.pack(padx=20,pady=20)
    text1="I = ("+ str(D)+" - "+str(R)+") /"+ str(R) + " = "+ str(PetitJeanIndex)
    label =tk.Label(root,text=text1,font=("Arial",18))
    label.pack(pady=30)
    fig, ax = plt.subplots(figsize=(15, 15))
    pos = nx.kamada_kawai_layout(G,scale=5)
    nx.draw(G, pos, with_labels=False, node_color='blue', node_size=500,ax=ax)
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas_widget = canvas.get_tk_widget()
    canvas_widget.pack()
    root.mainloop()

**Now, by applying all of these function, we move onto actually calculating the Petitjean index for a mol file**

1. First we use the `extract_mol()` function to extract the useful data found in the file. We get the bonds as an arrray, the CHEMBL ID of the molecule, and the value of the number of atoms. 
1. Now we can construct the graph **G** using the `contstruct_graph(bonds)` function taking as input the previously returned **bonds** array. 
1. This, along with the atoms count, enables us to construct the matrix **matrix** that will store all shortest paths in the graph. We are able to do this using the `shortest_path(G,atomCount)` function. It is important to note here that we need to increment the number of atoms by one as we have to keep in mind that no atom is numbered 0.
1. Furthermore, we use this matrix to get the list of eccentricities **eccentricities** using the `longest_shortest_paths(matrix)` function. 
1. Now, all we have to do is to get the diameter and the radius from this array using the `find_D_R(ecccentricities)` function. 

Once all of this is done we can simply apply the Formula **(D-R)/R** to get the Petitjean Index. 

Onto the display, we will be printing the ID of the molecule, followed by the petitjean index and the graphical representation of the molecule. 

The Graphical Representation of the atoms and bonds as nodes and edges will be displayed using the `GUI(G,D,R,Id,PetitJeanIndex)` function.

In [ ]:
bonds,Id,atomCount=extract_mol()
G=construct_graph(bonds)
matrix=shortest_path(G,atomCount+1)  
eccentricities=longest_shortest_paths(matrix)
D,R=find_D_R(eccentricities)
PetitJeanIndex= (D-R)/R

print(Id)

print("PetitJean Index= (",D,"-",R,")/",R,"=", PetitJeanIndex)

GUI(G,D,R,Id,PetitJeanIndex)


<font color=green size=5px>**Part 2**</font>
<div style='text-align:justify; font-size:16px'>

In the previous part, we covered the extraction of the structure as well as displaying it in a graph form and the calculation of its Petitjean index from a molfile. 
***
In the following, we will do the same, but for an SDF format type of file. 
 
We start by defining the function `extract_sdf()`. This function will extract the necessary infromation about the molecules (bonds, IDs, atom count).
It works in the following way:
* First we initialize empty lists to store the atom count of each molecule in the file, as well as the array of bonds for each molecule and its ID. 
* Next, the user will be asked to enter a valid path for the SDF file they wish to use. 
* The SDF file entered by the user will be opened and read as a list of lines **lines**
* Now, we will initialize an additional array and variables to store the information about the bonds, line count (keeps track of the current line number), atom count, and bond count. 

Next we will use a *for loop* to iterate through the lines of our SDF file.\ 
Each line will be stripped of the whitespaces at its beginning and its end. 
Then, it will be split into a list of elements. 

In the loop, we need to check if the line begins with four consecutive \$ characters. These will indicate that we moved from one molecule to another.\
So, the information about the previous molecule will be added to the **bonds** list by appending the moleculeBonds array to it.\
Then, we reset the **moleculeBonds** array to store the bonds of the next molecule. Now all that is left to do is reset the lineCount to 0 as well. Continue is used to skip the line lineCount+=1 as the lineCount of the next molecule should start by 0 and not 1.

When we get to the line which contains the counts of atoms and bonds (lineCount==3), we extract the values and store them respectively in the variables **atomCount** and **bondCount**. Next we will add the atomCount and append it to the **atomCounts** list. 

Now, onto the section where the bond information is stored (between line 4 and the line (4+atomCount+bondCount)). Each of the lines in this section of the file will contain two the number of two atoms that are connected by a bond. These will be stored as integers in the tuple **(atom1, atom2)**. Next, this bond information will be appended to the **moleculeBonds** list. 

It is important to note that in an sdf file, the lines that start with the string *"CHEMBL"* are the ones that contain the ID of the molecule, similiarily to what we have previously seen in the mol file section. Once this string is encountered, it will be used as a signal to store the ID of the molecule in the **moleculeId** variable. Each ID will be added to the **Ids** list. 

NB: After each line is processed, the lineCount is incremented by 1. 

After everything is done, the function will return the arrays containing the information about the Bonds, IDs, and atom counts, stored respectively in **bonds**, **Ids**, and **atomCounts**, for all the molecules in the SDF file. 

In [ ]:
def extract_sdf():
    
    atomCounts=[] 
    bonds=[]  
    Ids=[]  
    sdf_path=input("Enter the sdfile path: ")
    with open(sdf_path,'r') as sd_file:
        lines=sd_file.read().splitlines()
        
        moleculeBonds=[]  
        lineCount=0  
        atomCount=0
        bondCount=0
        for line in lines:
            elements=line.strip().split()
            if line.startswith("$$$$"):
                bonds.append(moleculeBonds)
                moleculeBonds=[]  
                lineCount=0 
                continue  
            elif lineCount==3:
                atomCount=int(elements[0].strip())
                bondCount=int(elements[1].strip())
                atomCounts.append(atomCount)
            elif lineCount>3+atomCount and lineCount<=3+atomCount+bondCount:
                atom1=int(elements[0].strip())
                atom2=int(elements[1].strip())
                moleculeBonds.append((atom1,atom2))
            elif line.startswith("CHEMBL"):
                moleculeId=line
                Ids.append(moleculeId)
            lineCount+=1
                
    return bonds,Ids,atomCounts

**Now, by applying the preceeding functions, we move onto actually calculating the Petitjean index for an SDF file**

First, we use the `extract_sdf()` function to retrieve the necessary information from each molecule, which will be stored in the **bonds**, **Ids**, and **atomCounts** arrays. 

Next, we iterate over each molecule:
* First, we get the atom count (remember that we increment by one because indexing starts from 0).
* Next, we will construct a graph for the molecule whose bonds tuples are stored in position i in the bonds array. This will be done using the `construct_graph(bonds[i])` function. 
* Now, we calculate the eccentricities using the `longest_shortest_paths(matrix)` function. These will be used to get the radius and the diameter by being used as inputs for the `find_D_R(eccentricities)` function.
* Just like we did for the mol files, we will simply use the Radius and Diameter to calculate the petitJean index using the corresponding formula ((D-R)/R). 

All that is left to do is simply print the molecule ID for each molecule as well as the calculated Petitjean Index. 

The function `GUI(G)` will be used to visualize the graph G representing the structure of every molecule. 


In [ ]:
bonds,Ids,atomCounts=extract_sdf()
for i in range(len(bonds)):
    atomCount=atomCounts[i]+1
    G=construct_graph(bonds[i])
    matrix=shortest_path(G,atomCount)
    eccentricities=longest_shortest_paths(matrix)
    
    D,R=find_D_R(eccentricities)
    PetitJeanIndex= (D-R)/R
    print(Ids[i])
    print("PetitJean Index= (",D,"-",R,")/",R,"=", PetitJeanIndex,end='\n\n')
    GUI(G,D,R,Ids[i],PetitJeanIndex)

# END